In [1]:
import json
import numpy as np
import pandas as pd
import requests
import urllib
from PIL import Image
import io
from io import BytesIO

In [2]:
data = pd.read_json('Indian_Number_plates.json', lines=True)

In [3]:
data.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN


## Downloading images 

In [12]:
def convertPoints(row):
    data = row["annotation"]
    topx = data[0]["points"][0]["x"]
    topy = data[0]["points"][0]["y"]
    bottomx =data[0]["points"][1]["x"]
    bottomy =data[0]["points"][1]["y"]
    
    return topx, topy, bottomx, bottomy
    
    

In [15]:
def downloadData(df, outdir):
    counter = 0
    imagenames = []
    topxs = []
    topys = []
    bottomxs = []
    bottomys = []
    for index, row in df.iterrows():
        
        response = requests.request("GET", row['content'])
        im = Image.open(BytesIO(response.content))
        im = im.convert('RGB')
#         print(outdir + "/licensed_car{}.jpeg".format(counter))
        counter  += 1
        im.save(outdir + "/car{}.jpeg".format(counter))
        imagenames.append(outdir + "/car{}.jpeg".format(counter))
        data = row["annotation"]
        width = data[0]["imageWidth"]
        height = data[0]["imageHeight"]
        topx, topy, bottomx, bottomy = convertPoints(row)
        topxs.append(int(round(topx*width)))
        topys.append(int(round(topy*height)))
        bottomxs.append(int(round(bottomx*width)))
        bottomys.append(int(round(bottomy*height)))
    print('Downloaded {} images'.format(counter))
    return imagenames, topxs, topys, bottomxs, bottomys

In [16]:
imgNames, topxs, topys, bottomxs, bottomys = downloadData(data, 'Images')

Downloaded 237 images


In [18]:
data_final = pd.DataFrame()
data_final['imagenames'] = imgNames
data_final['topx'] = topxs
data_final['topy'] = topys
data_final['bottomx'] = bottomxs
data_final['bottomy'] = bottomys


In [19]:
data_final.head()

,imagenames,topx,topy,bottomx,bottomy
0,Images/car1.jpeg,582,274,700,321
1,Images/car2.jpeg,80,382,288,449
2,Images/car3.jpeg,41,219,115,260
3,Images/car4.jpeg,217,313,359,359
4,Images/car5.jpeg,209,220,866,554


In [20]:
data_final.to_csv('traindata.csv', index=False)